In [1]:
import json
import pandas as pd
from features import domain_tld_extract
from utils import *
from joblib import load
from features import dga_classifier

In [2]:
data = []

with open("../data/large_eve.json",'r',encoding = 'utf-8') as f:
    for i in f:
        if i:
            data.append(json.loads(i))


print("Total Data: ", len(data))
f.close()

Total Data:  746909


In [3]:
'''DNS Register '''
dns = [d for d in data if "dns" in d.keys()]

print("DNS Register Data Quantity: ", len(dns))

DNS Register Data Quantity:  15749


In [4]:
'''DNS Data'''
print(dns[0:2])

[{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}}, {'timestamp': '2017-07-22T17:33:24.990320-0500', 'flow_id': 2022925111925872, 'pcap_cnt': 54352, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 38051, 'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 58278, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}]


In [5]:
df = pd.json_normalize(dns)
df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


In [6]:
'''A Type DNS Filtering'''
filtered_df = df[df['dns.rrtype'] == 'A']
print("A Type DNS: ", len(filtered_df))

A Type DNS:  2849


In [7]:
unique_df = filtered_df.drop_duplicates(subset=['dns.rrname'])
print("Unique A Type DNS:", len(unique_df))
unique_df.head()

Unique A Type DNS: 177


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN


In [8]:
'''Unique TLD Registers'''
unique_df['tld_flare'] = unique_df['dns.rrname'].apply(domain_tld_extract)

<ipython-input-8-3f4911ac3a08>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['tld_flare'] = unique_df['dns.rrname'].apply(domain_tld_extract)


In [9]:
unique_df['tld_flare']

nan_value = float("NaN")
unique_df.replace("", nan_value, inplace=True)

unique_df.dropna(subset = ["tld_flare"], inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-9-19046e3b40fc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df.dropna(subset = ["tld_flare"], inplace=True)


In [10]:
for i in unique_df['tld_flare']:
    print(i)

wunderground.com
dropbox.com
aoltw.net
safebrowsing.clients.google.com.home
mozilla.com
metasploit.com
aol.com
aoltw.net
aol.com
aol.com
aoltw.net
aol.com
google.com
wpad.home
stayonline.net
aoltw.net
aol.com
aol.com
192.168.22.110phpmyadmin
informaction.com
secure.informaction.com.localdomain
safebrowsing.clients.google.com.localdomain
vmware.com
mozilla.com
192.168.22.110phpmyadmin.localdomain
windows.com
vmware.com
ntkrnlpa.info
portswigger.net
offensive-security.com
stayonline.net
stopbadware.org
aoltw.net
arrancar.org
sql-ledger.org
backtrack-linux.org
stayonline.net
mozilla.com
theanime.cn
theanime.cn
aoltw.net
aoltw.net
aoltw.net
safebrowsing.clients.google.com.hackerlabs.vpn
stayonline.net
aoltw.net
aol.com
aoltw.net
phpmyadmin.net
google.com
google.com
microsoft.com
secure.informaction.com.home
aoltw.net
comcast.net
google.com
aoltw.net
ubuntu.com
mozilla.com
alexa.com
postgresql.org
sourceforge.net
freepbx.org
secure.informaction.com.hackerlabs.vpn
bigflickrfeed.com
gnu.org
w

In [11]:
feature_df = pd.DataFrame()
urls = unique_df['tld_flare']
feature_df['length'] =  unique_df['tld_flare'].str.len()
feature_df['digits'] = unique_df['tld_flare'].str.count('[0-9]')
feature_df['entropy'] = unique_df['tld_flare'].apply(H_entropy)
feature_df['VocalsConsonants'] = unique_df['tld_flare'].apply(VocalsConsonants)
feature_df['firstDigitPos'] = unique_df['tld_flare'].apply(firstDigitPos)

In [12]:
feature_df

,length,digits,entropy,VocalsConsonants,firstDigitPos
0,16,0,3.375000,0.500000,0
1,11,0,3.027169,0.428571,0
2,9,0,2.947703,0.600000,0
6,36,0,3.861052,0.600000,0
9,11,0,2.913977,0.666667,0
...,...,...,...,...,...
15632,26,0,3.584303,0.642857,0
15661,13,0,3.026987,0.500000,0
15701,12,0,3.418296,0.571429,0
15716,30,11,3.802910,0.625000,1


# Part 2

In [13]:
clf = load('model.joblib')
predictions = clf.predict(feature_df)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [14]:
print("Predictions from Type A DNS:  ")
print(predictions)

Predictions from Type A DNS:  
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 0 0 1 0]


In [15]:
filter_registers = [False if i == 0 else True for i in predictions ]

In [16]:
print("Important Registers: ")
for i in unique_df[filter_registers]['tld_flare']:
    print(i)

Important Registers: 
safebrowsing.clients.google.com.home
192.168.22.110phpmyadmin
safebrowsing.clients.google.com.localdomain
192.168.22.110phpmyadmin.localdomain
ntkrnlpa.info
portswigger.net
sql-ledger.org
backtrack-linux.org
safebrowsing.clients.google.com.hackerlabs.vpn
phpmyadmin.net
postgresql.org
secure.informaction.com.hackerlabs.vpn
safebrowsing.clients.google.com.lan
"192.168.206.56"
windowsupdate.com
192.168.26-27.0
fastserv.com
liquidweb.com
fastserv.com
team-cymru.org
192.168.21.1201
msftncsi.com
msftncsi.com
vtlfccmfxlkgifuf.com
1922.168.22.254
1922.168.22.254.home
192.168.21-28.0
ejfodfmfxlkgifuf.xyz
192.168.21-28.0.home
192.168.22.201:
192.168.22.201:.stayonline.net


In [17]:
dga = dga_classifier()

[*] Initializing... training classifier - Please wait.
[+] Training classifier on training set


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /


[+] Out of sample legit f1 score 0.9988452453239457
[+] Out of sample dga f1 score 0.588235294117647
[+] Training final classifier


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /


[+] Classifier Ready


In [18]:
unique_df['good'] = unique_df['tld_flare'].apply(dga.predict)

<ipython-input-18-7788ed5be64b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['good'] = unique_df['tld_flare'].apply(dga.predict)


In [19]:
unique_df[unique_df['good'] == 'dga']['tld_flare']

7963             msftncsi.com
8026             msftncsi.com
9856     vtlfccmfxlkgifuf.com
10048    1922.168.22.254.home
10646    ejfodfmfxlkgifuf.xyz
Name: tld_flare, dtype: object

# Are them the same  domain as the model?
Somehow yes. Let's say that the model contains the DGA Clasificator, as thise one classified 4 as DGA and the class model has around 30. We also see there's a couple of false positives, like windowsupdate.com or postgresql.org.


## Parte 3

In [20]:
from umbrella import *

In [21]:
umbrellaCisco = Umbrella(limit= 1000000)

[+] Updated Umbrella Top 1 Million list...


In [22]:
out_umbrella = [False if umbrellaCisco.domain_in_umbrella(i) else True for i in unique_df['tld_flare'] ]

In [23]:
purged_df = unique_df[out_umbrella]

In [24]:
purged_df

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld_flare,good
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net,legit
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home,legit
21,2017-07-22T17:33:27.407602-0500,770491173779506,54535,dns,150,192.168.205.170,15001,192.168.207.4,53,UDP,query,38706,aolmtcmxm02.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net,legit
39,2017-07-22T17:33:27.389328-0500,742977613328592,54526,dns,150,192.168.205.170,9876,192.168.207.4,53,UDP,query,22415,aolmtcmxm03.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net,legit
51,2017-07-22T17:34:03.616631-0500,1718592321841335,57618,dns,180,192.168.198.62,44114,192.168.207.4,53,UDP,query,55738,wpad.home,A,0.0,NaN,NaN,NaN,wpad.home,legit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14730,2017-07-22T19:27:13.026377-0500,919600000755465,3238844,dns,180,192.168.198.62,49001,192.168.207.4,53,UDP,query,8420,192.168.21-28.0.home,A,0.0,NaN,NaN,NaN,192.168.21-28.0.home,legit
15507,2017-07-22T19:39:02.425944-0500,487638006398936,4319590,dns,180,192.168.198.58,53630,192.168.207.4,53,UDP,query,8127,192.168.22.201:,A,0.0,NaN,NaN,NaN,192.168.22.201:,legit
15540,2017-07-22T19:42:23.423583-0500,1521234784253599,4375549,dns,150,192.168.205.170,1295,192.168.207.4,53,UDP,query,36694,aoldtcmds01.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net,legit
15632,2017-07-22T19:44:29.056834-0500,787856241843714,4405786,dns,130,192.168.203.69,60088,192.168.207.4,53,UDP,query,10109,ntp.ubuntu.com.localdomain,A,0.0,NaN,NaN,NaN,ntp.ubuntu.com.localdomain,legit


In [25]:
for i in unique_df['tld_flare']:
    if umbrellaCisco.domain_in_umbrella(i): print(i,'Out of umbrella')
    else:print(i,'Into umbrella')

wunderground.com Out of umbrella
dropbox.com Out of umbrella
aoltw.net Into umbrella
safebrowsing.clients.google.com.home Into umbrella
mozilla.com Out of umbrella
metasploit.com Out of umbrella
aol.com Out of umbrella
aoltw.net Into umbrella
aol.com Out of umbrella
aol.com Out of umbrella
aoltw.net Into umbrella
aol.com Out of umbrella
google.com Out of umbrella
wpad.home Into umbrella
stayonline.net Into umbrella
aoltw.net Into umbrella
aol.com Out of umbrella
aol.com Out of umbrella
192.168.22.110phpmyadmin Into umbrella
informaction.com Out of umbrella
secure.informaction.com.localdomain Into umbrella
safebrowsing.clients.google.com.localdomain Into umbrella
vmware.com Out of umbrella
mozilla.com Out of umbrella
192.168.22.110phpmyadmin.localdomain Into umbrella
windows.com Out of umbrella
vmware.com Out of umbrella
ntkrnlpa.info Into umbrella
portswigger.net Out of umbrella
offensive-security.com Out of umbrella
stayonline.net Into umbrella
stopbadware.org Out of umbrella
aoltw.ne

In [26]:
purged_df[['tld_flare','good']]

,tld_flare,good
2,aoltw.net,legit
6,safebrowsing.clients.google.com.home,legit
21,aoltw.net,legit
39,aoltw.net,legit
51,wpad.home,legit
...,...,...
14730,192.168.21-28.0.home,legit
15507,192.168.22.201:,legit
15540,aoltw.net,legit
15632,ntp.ubuntu.com.localdomain,legit


In [27]:
for i in unique_df[unique_df['good'] == 'dga']['tld_flare']:
    if umbrellaCisco.domain_in_umbrella(i):print(i, 'Into umbrella')
    else:print(i,'Outside umbrella')

msftncsi.com Into umbrella
msftncsi.com Into umbrella
vtlfccmfxlkgifuf.com Outside umbrella
1922.168.22.254.home Outside umbrella
ejfodfmfxlkgifuf.xyz Outside umbrella


In [28]:
for i in unique_df[unique_df['good'] == 'dga']['tld_flare']:
    if umbrellaCisco.domain_tld_in_umbrella(i):print(i, 'Into umbrella')
    else:print(i,'Outside umbrella')

msftncsi.com Into umbrella
msftncsi.com Into umbrella
vtlfccmfxlkgifuf.com Outside umbrella
1922.168.22.254.home Outside umbrella
ejfodfmfxlkgifuf.xyz Outside umbrella


In [29]:
unique_df[['tld_flare','good']]

,tld_flare,good
0,wunderground.com,legit
1,dropbox.com,legit
2,aoltw.net,legit
6,safebrowsing.clients.google.com.home,legit
9,mozilla.com,legit
...,...,...
15632,ntp.ubuntu.com.localdomain,legit
15661,microsoft.com,legit
15701,verisign.com,legit
15716,192.168.22.201:.stayonline.net,legit


In [30]:
import whois

In [37]:
for i in unique_df[unique_df['good'] == 'dga']['tld_flare']:
    try:
        date= whois.whois(i).creation_date
        print(unique_df[['tld_flare']], date)
    except:
        pass

                                  tld_flare
0                          wunderground.com
1                               dropbox.com
2                                 aoltw.net
6      safebrowsing.clients.google.com.home
9                               mozilla.com
...                                     ...
15632            ntp.ubuntu.com.localdomain
15661                         microsoft.com
15701                          verisign.com
15716        192.168.22.201:.stayonline.net
15737                              real.com

[176 rows x 1 columns] [datetime.datetime(2005, 11, 10, 22, 6, 51), datetime.datetime(2005, 11, 10, 17, 6, 51)]
                                  tld_flare
0                          wunderground.com
1                               dropbox.com
2                                 aoltw.net
6      safebrowsing.clients.google.com.home
9                               mozilla.com
...                                     ...
15632            ntp.ubuntu.com.localdomain
15661  

In [36]:
for i in unique_df[unique_df['good'] == 'dga']['tld_flare']:
    try:
        date=whois.whois(i).creation_date
        print(date)
    except:
        pass

[datetime.datetime(2005, 11, 10, 22, 6, 51), datetime.datetime(2005, 11, 10, 17, 6, 51)]
[datetime.datetime(2005, 11, 10, 22, 6, 51), datetime.datetime(2005, 11, 10, 17, 6, 51)]
None
None
